# Univariate Analysis 

The main objectives of this notebook are: 
- kj

In [1]:
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt

In [13]:
df = pl.read_csv("../data/events.csv")

In [14]:
df.head()

UserId,ProductId,Rating,Timestamp
str,str,f64,i64
"""A39HTATAQ9V7YF""","""0205616461""",5.0,1369699200
"""A3JM6GV9MNOF9X""","""0558925278""",3.0,1355443200
"""A1Z513UWSAAO0F""","""0558925278""",5.0,1404691200
"""A1WMRR494NWEWV""","""0733001998""",4.0,1382572800
"""A3IAAVS479H7M7""","""0737104473""",1.0,1274227200


In [4]:
df_clean = df.drop_nulls()

In [5]:
brand = df['brand'].value_counts(sort=True)
brand = brand[1:]
brand

brand,count
str,u32
"""samsung""",7889245
"""apple""",6259379
"""xiaomi""",4638062
"""huawei""",1410126
"""lucente""",1185075
…,…
"""yoobao""",1
"""build-a-bear""",1
"""viga""",1


In [6]:
purchase_df = df.filter(pl.col('event_type') == 'purchase')
purchase_df.tail()

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
str,str,i64,i64,str,str,f64,i64,str
"""2019-11-30 23:58:14 UTC""","""purchase""",1004874,2053013555631882655,"""electronics.smartphone""","""samsung""",346.7,547804983,"""717566cf-ef93-4078-ba8f-169a3a…"
"""2019-11-30 23:58:22 UTC""","""purchase""",1005130,2053013555631882655,"""electronics.smartphone""","""apple""",1437.02,515582054,"""829c20b5-696e-4a8a-8a9f-171014…"
"""2019-11-30 23:58:57 UTC""","""purchase""",1004767,2053013555631882655,"""electronics.smartphone""","""samsung""",235.6,579876821,"""ca50e291-43f3-4ca2-9e13-20ee6b…"
"""2019-11-30 23:59:15 UTC""","""purchase""",3701309,2053013565983425517,"""appliances.environment.vacuum""","""polaris""",89.32,543733099,"""a65116f4-ac53-4a41-ad68-660678…"
"""2019-11-30 23:59:44 UTC""","""purchase""",26205399,2053013563693335403,null,null,179.16,553345124,"""58c692ff-c7a9-4e35-9ec4-58598f…"


In [7]:
unique_purchases = purchase_df.select(pl.col('user_session').n_unique().alias("unique"))
print(unique_purchases)

shape: (1, 1)
┌────────┐
│ unique │
│ ---    │
│ u32    │
╞════════╡
│ 773214 │
└────────┘


In [8]:
purchase_df.select(pl.col('user_session').is_duplicated())

user_session
bool
false
false
false
false
true
…
true
false
true


In [12]:
purchase_df.select(pl.col('user_session').is_unique())

user_session
bool
true
true
true
true
false
…
false
true
false
